In [1]:
!pip -q install -U diffusers transformers accelerate soundfile torchsde huggingface_hub

from huggingface_hub import notebook_login
notebook_login()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 32.6 MB/s eta 0:00:00


In [2]:
import os
import torch
import soundfile as sf
from diffusers import StableAudioPipeline

pipe = StableAudioPipeline.from_pretrained(
    "stabilityai/stable-audio-open-1.0",
    torch_dtype=torch.float16
).to("cuda")

os.makedirs("sounds", exist_ok=True)

# 4 distinct prompts + different durations
jobs = [
    ("sound-01.wav", 3.0,  "tiny jungle insect chatter, fast clicks, close mic, no music"),
    ("sound-02.wav", 6.0,  "wet swamp frog croak, field recording ambience, no music"),
    ("sound-03.wav", 10.0, "large fictional beast call, resonant roar, distant canopy echo, no music"),
    ("sound-04.wav", 14.0, "haunting whistle creature call, leaves rustling, branches snapping, no music"),
]

negative_prompt = "music, vocals, low quality"

for i, (fname, seconds, prompt) in enumerate(jobs):
    gen = torch.Generator("cuda").manual_seed(2000 + i)

    audio = pipe(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=200,
        audio_end_in_s=seconds,
        num_waveforms_per_prompt=1,
        generator=gen,
    ).audios[0]  # (channels, samples)

    wav = audio.T.float().cpu().numpy()  # (samples, channels)
    sf.write(f"sounds/{fname}", wav, pipe.vae.sampling_rate)

print("Saved files:", os.listdir("sounds"))


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


model_index.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/99 [00:00<?, ?it/s]

...

/usr/local/lib/python3.12/dist-packages/torchsde/_brownian/brownian_interval.py:608: UserWarning: Should have tb<=t1 but got tb=500.00006103515625 and t1=500.0.
  warnings.warn(f"Should have {tb_name}<=t1 but got {tb_name}={tb} and t1={self._end}.")
/usr/local/lib/python3.12/dist-packages/torchsde/_brownian/brownian_interval.py:599: UserWarning: Should have ta>=t0 but got ta=0.29999998211860657 and t0=0.3.
  warnings.warn(f"Should have ta>=t0 but got ta={ta} and t0={self._start}.")
/usr/local/lib/python3.12/dist-packages/torchsde/_brownian/brownian_interval.py:599: UserWarning: Should have ta>=t0 but got ta=0.0 and t0=0.3.
  warnings.warn(f"Should have ta>=t0 but got ta={ta} and t0={self._start}.")
/usr/local/lib/python3.12/dist-packages/torchsde/_brownian/brownian_interval.py:602: UserWarning: Should have tb>=t0 but got tb=0.29999998211860657 and t0=0.3.
  warnings.warn(f"Should have {tb_name}>=t0 but got {tb_name}={tb} and t0={self._start}.")


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

Saved files: ['sound-04.wav', 'sound-02.wav', 'sound-03.wav', 'sound-01.wav']


In [3]:
import shutil
shutil.make_archive("sounds", "zip", "sounds")
print("Created sounds.zip")


Created sounds.zip
